In [ ]:
!pip install beauty-print pydantic more-itertools
!pip3 install -U https://github.com/pyrogram/pyrogram/archive/master.zip

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
     - 711 kB 3.6 MB/s
  Created wheel for Pyrogram: filename=Pyrogram-1.1.0b1-py3-none-any.whl size=2714976 sha256=5efd32da8f65e0d9e578ff190b30804d2f968bd43510be130e5e3144d5ce41a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-t7gwlcys/wheels/35/26/65/c4b8034f6a2df9cffbd3709aa83a95db39b4beb975f2bdbcf2
Successfully built Pyrogram
  Attempting uninstall: Pyrogram
    Found existing installation: Pyrogram 1.1.0b1
    Uninstalling Pyrogram-1.1.0b1:
      Successfully uninstalled Pyrogram-1.1.0b1
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [ ]:
from common import client

if client.is_connected:
    print("Already connected.")
else:
    await client.start()
    print("Connected, client started!")

Pyrogram v1.1.0b1, Copyright (C) 2017-2020 Dan <https://github.com/delivrance>
Licensed under the terms of the GNU Lesser General Public License v3 or later (LGPLv3+)

Connected, client started!


### Modeling

Let's define some models and helper functions

In [ ]:
from utils.message_entities import parse_entities
from typing import *
from collections import defaultdict
from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic import validator
from bprint import bprint
from utils.names import user_or_display_name
import json
from pyrogram.raw.functions.messages import GetReplies

def pydantic_dump(obj: Any):
    from pydantic.json import pydantic_encoder
    print(json.dumps(obj, indent=4, default=pydantic_encoder))

    
class UserModel(BaseModel):
    first_name: Optional[str]
    last_name: Optional[str]
    username: Optional[str]
        
    class Config:
        orm_mode = True
    

class MessageModel(BaseModel):
    message_id: int = Field(..., alias="id")
    from_user: Optional[UserModel]
    text: str = Field(..., alias="message")
    reply_to_message: Optional["MessageModel"] = None
    
    class Config:
        orm_mode = True
        arbitrary_types_allowed = True
        allow_population_by_field_name = True
    
    def get_link(self) -> str:
        return f"https://t.me/botlistchat/{self.message_id}"
    
    @validator('text', pre=True)
    def strip(cls, value):
        return value.replace("\n\n", "\n")

MessageModel.update_forward_refs()
    

class ReplyChain(BaseModel):
    reference: MessageModel
    bots: List[str]
    replies: List[MessageModel]

        
def parse_msg(message):
    return MessageModel(**json.loads(str(message)))

import asyncio
    
async def get_replies(message, should_include: Callable):
    response = await client.send(
        GetReplies(
            peer=await client.resolve_peer("botlistchat"),
            msg_id=message.message_id,
            offset_id=0,
            offset_date=0,
            add_offset=0,
            limit=100,
            max_id=9999999,
            min_id=0,
            hash=0,
        )
    )
    print("Num replies:", len(response.messages))
    replies = [parse_msg(x) for x in response.messages]
    
    await asyncio.sleep(1)
    
    return [x for x in replies if should_include(x)]
    
    """
    replies = []
    
    curr = message.reply_to_message
    
    while curr:
        if should_include(curr):
            replies.append(parse_msg(curr))
        curr = curr.reply_to_message
        
        if not curr: 
            pass
    
    return replies
    """


In [ ]:
def should_include_message(msg: MessageModel) -> bool:
    text = msg.text
    if not text:
        return False
    
    if len(text) < 5:
        return False
    
    return True

chains = []

async for m in client.iter_history("botlistchat", limit=300):
    ent = parse_entities(m, 'mention')
    
    bots_in_message = []
    
    for e in ent:
        if e.text.lower().endswith("bot"):
            bots_in_message.append(e.text)
    
    if not bots_in_message:
        continue
        
    replies = list(reversed(list(await get_replies(m, should_include_message))))
    
    if not replies:
        continue
    
    chains.append(ReplyChain(reference=parse_msg(m), bots=bots_in_message, replies=replies))


for c in chains:
    for reply in c.replies:
        print(f"{user_or_display_name(reply.from_user)} 👤> {reply.text}")
        # print(f"> {reply.text}")
        print(reply.get_link())
        
    for bot in c.bots:
        print(f"--> {bot}")
    
    print()


Num replies: 5
Num replies: 5
Num replies: 18
Num replies: 0
Num replies: 0
Num replies: 11
Num replies: 1
Num replies: 0
Num replies: 4
Num replies: 4
Num replies: 1
Num replies: 4
Num replies: 0
Num replies: 0
Num replies: 0
Num replies: 3
Num replies: 3
Num replies: 1
Num replies: 2
Num replies: 3
Num replies: 2
Num replies: 13


AttributeError: 'NoneType' object has no attribute 'username'